In [8]:
import os
import json
import openai
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import cv2
import numpy as np
from PIL import Image

In [9]:
import openai

# OpenAI 클라이언트 생성
client = openai.OpenAI()

def fix_ocr_text(ocr_text):
    """ OCR로 추출한 텍스트를 GPT-3.5 Turbo로 정리 """
    prompt = f"""
    다음은 OCR을 통해 추출한 텍스트입니다.  
    오류를 수정하고, 문장을 자연스럽게 정리해 주세요.  
    - 문법 및 맞춤법 오류 수정  
    - 한국어 & 영어 혼합 문제 해결  
    - OCR 오류로 인한 기호 삭제  

    📄 OCR 원본:
    {ocr_text}

    ✏ 정리된 결과를 반환해주세요.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a text correction assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# 고등사회문제 json파일 만들기

In [10]:
import json
import pdfplumber
import pytesseract
import os
import re
from pdf2image import convert_from_path
from PIL import Image

In [18]:
# 📌 폴더의 경로 가져오기
base_dir = os.path.abspath(os.path.join(os.getcwd(), "./data"))
folder_path = os.path.join(base_dir, "사회")  # "data/사회"를 올바르게 설정

In [19]:
# 📌 텍스트 정리 함수 (공백/개행 정리)
def clean_text(text):
    text = text.replace("\n", " ").strip()  # 개행을 공백으로 변경
    text = re.sub(r"\s+", " ", text)  # 여러 개의 공백을 하나로
    return text

In [20]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        combined_text_list = []

        for page in pdf.pages:
            width, height = page.width, page.height

            # 왼쪽 문항 (페이지별 왼쪽 먼저)
            left_bbox = (0, 0, width / 2, height)
            left_crop = page.within_bbox(left_bbox)
            left_text = left_crop.extract_text()
            if left_text:
                combined_text_list.append(clean_text(left_text))  # 정리 후 추가

            # 오른쪽 문항 (페이지별 오른쪽 나중)
            right_bbox = (width / 2, 0, width, height)
            right_crop = page.within_bbox(right_bbox)
            right_text = right_crop.extract_text()
            if right_text:
                combined_text_list.append(clean_text(right_text))

    return "\n".join(combined_text_list)  # 왼쪽 → 오른쪽 순서 유지

In [21]:
# 📌 OCR 이미지 PDF 처리 (페이지별 왼쪽 → 오른쪽)
def extract_text_from_image_pdf(pdf_path):
    images = convert_from_path(pdf_path, dpi=300)
    combined_text_list = []

    for img in images:
        width, height = img.size

        # 왼쪽 문항 OCR
        left_crop = img.crop((0, 0, width // 2, height))
        left_text = pytesseract.image_to_string(left_crop, lang="eng+kor", config="--psm 6")
        combined_text_list.append(clean_text(left_text))

        # 오른쪽 문항 OCR
        right_crop = img.crop((width // 2, 0, width, height))
        right_text = pytesseract.image_to_string(right_crop, lang="eng+kor", config="--psm 6")
        combined_text_list.append(clean_text(right_text))

    return "\n".join(combined_text_list)

# 📌 OCR 후 텍스트 정리
def fix_ocr_text(text):
    return clean_text(text)

In [22]:
# 📌 파일명 정렬 (연도_회차로 정렬)
def extract_numbers(filename):
    # 연도와 회차를 추출하는 정규식
    match = re.match(r"(\d{4})_.*?(\d+)(?:차)", filename)
    if match:
        return int(match.group(1)), int(match.group(2))  # 연도, 회차 변환
    return (0, 0)

In [24]:
# 📌 JSON 파일로 변환 및 저장
def save_processed_text_to_json(folder_path, output_json):
    all_data = {}
    pdf_files = sorted(
        [f for f in os.listdir(folder_path) if f.endswith(".pdf")],
        key=extract_numbers
    )

    for file in pdf_files:
        pdf_path = os.path.join(folder_path, file)
        print(f"📄 Processing: {file}")

        # 1️⃣ PDF에서 직접 텍스트 추출
        raw_text = extract_text_from_pdf(pdf_path)

        # 2️⃣ OCR 적용 (텍스트가 없으면 OCR 사용)
        if not raw_text.strip():
            raw_text = extract_text_from_image_pdf(pdf_path)

        # 3️⃣ OCR 정리
        cleaned_text = fix_ocr_text(raw_text)

        # 4️⃣ 데이터 저장
        all_data[file] = {
            "raw_text": raw_text,
            "cleaned_text": cleaned_text
        }

    # ✅ JSON 파일로 저장
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print(f"✅ 모든 PDF 파일이 {output_json}에 저장되었습니다!")

In [27]:
# 🚀 실행
save_processed_text_to_json(folder_path, f'{base_dir}/society_questions.json')

📄 Processing: 2018_고졸학력_1차시험_사회.pdf
📄 Processing: 2018_고졸학력_2차시험_사회.pdf
📄 Processing: 2019_고졸학력_1차시험_사회.pdf
📄 Processing: 2019_고졸학력_2차시험_사회.pdf
📄 Processing: 2020_고졸학력_1차시험_사회.pdf
📄 Processing: 2020_고졸학력_2차시험_사회.pdf
📄 Processing: 2021_고졸학력_1차시험_사회.pdf
📄 Processing: 2021_고졸학력_2차시험_사회.pdf
📄 Processing: 2022_고졸학력_1차시험_사회.pdf
📄 Processing: 2022_고졸학력_2차시험_사회.pdf
📄 Processing: 2023_고졸학력_1차시험_사회.pdf
📄 Processing: 2023_고졸학력_2차시험_사회.pdf
📄 Processing: 2024_고졸학력_1차시험_사회.pdf
📄 Processing: 2024_고졸학력_2차시험_사회.pdf
✅ 모든 PDF 파일이 /Users/jungyeon/Desktop/encore/project/mini3/data/society_question.json에 저장되었습니다!


# 고등사회답지 json파일 만들기

In [28]:
import json
import pdfplumber
import re
import os
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

In [29]:
# 📌 한글 숫자 기호 변환 (① → 1, ② → 2, ③ → 3, ④ → 4)
def convert_korean_numbers(text):
    number_map = {"①": "1", "②": "2", "③": "3", "④": "4"}
    for k, v in number_map.items():
        text = text.replace(k, v)
    return text

In [30]:
# 📌 OCR 적용 (이미지 기반 PDF 처리)
def extract_text_from_image_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    text = "\n".join([pytesseract.image_to_string(img, lang="eng+kor") for img in images])
    return text

In [41]:
# 📌 PDF에서 사회 정답 추출 (OCR 포함)
def extract_english_answers_from_pdf(pdf_path):
    answers = {}

    # 1️⃣ PDF에서 직접 텍스트 추출
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

    # 2️⃣ OCR 적용 (텍스트가 비어 있으면 OCR 사용)
    if not text.strip():
        text = extract_text_from_image_pdf(pdf_path)

    # 3️⃣ OCR로 추출한 원본 텍스트 출력 (디버깅 목적)
    print("\n📝 OCR EXTRACTED TEXT FROM PDF:", pdf_path)
    print(text[:1000])  # 처음 1000자만 출력

    # 4️⃣ "사회 정답표" 또는 "4교시 사회" 포함된 부분 찾기
    match = re.search(r"(?:사회 정답표|4교시 사회|사회)([\s\S]+?)(?=\n\w+ 정답표|\Z)", text)
    if match:
        english_answers_section = match.group(1).strip()
    else:
        print(f"⚠ 사회 정답을 찾을 수 없음: {pdf_path}")
        return None

    # 5️⃣ 정답 패턴 추출 (디버깅용 출력 추가)
    extracted_text = convert_korean_numbers(english_answers_section)
    print("\n🔍 EXTRACTED ENGLISH ANSWERS SECTION:")
    print(extracted_text[:500])  # 처음 500자만 출력

    # 6️⃣ 문항번호 & 정답 추출
    answer_pattern = re.findall(r"(\d+)\s+([①②③④1-4])", extracted_text)

    # 🔥 디버깅: 추출된 정답 출력
    print("\n🔎 Extracted Answers Dictionary:", answer_pattern)

    for q_num, ans in answer_pattern:
        answers[q_num] = ans

    return answers

In [42]:
# 📌 모든 PDF에서 사회 정답 저장 (JSON 생성)
def save_society_answers_to_json(folder_path, output_json="./data/society_answers.json"):
    all_answers = {}

    pdf_files = sorted(
        [file for file in os.listdir(folder_path) if file.endswith(".pdf")],
        key=lambda x: (
            int(x.split("_")[0]),  # 연도 (2018)
            int(re.search(r'(\d+)차', x).group(1)) if re.search(r'(\d+)차', x) else 0  # 회차 (1차, 2차 등)
        )
    )

    for file in pdf_files:
        pdf_path = os.path.join(folder_path, file)

        print(f"\n📄 Processing Answer Sheet: {file}")

        answers = extract_english_answers_from_pdf(pdf_path)
        if answers:
            print(f"\n✅ Final Answers Before JSON Save ({file}):", answers)
            all_answers[file] = answers
        else:
            print(f"⚠ 사회 정답이 없음: {file}")

    # 📌 JSON 파일 저장
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_answers, f, ensure_ascii=False, indent=4)

    print(f"✅ 사회 정답 JSON 저장 완료! ({output_json})")

In [43]:
# ✅ 실행 (폴더 경로 설정)
save_society_answers_to_json("./data/정답")


📄 Processing Answer Sheet: 2018_고졸학력_1차시험_정답.pdf

📝 OCR EXTRACTED TEXT FROM PDF: ./data/정답/2018_고졸학력_1차시험_정답.pdf
2018년도 제1회 고등학교 졸업학력 검정고시
1교시
국어 정답표
문항번호 정 답 문항번호 정 답 문항번호 정 답
1 ② 11 ① 21 ④
2 ③ 12 ④ 22 ①
3 ③ 13 ③ 23 ②
4 ① 14 ③ 24 ②
5 ③ 15 ④ 25 ①
6 ① 16 ②
7 ② 17 ④
8 ③ 18 ①
9 ① 19 ③
10 ④ 20 ②
2018년도 제1회 고등학교 졸업학력 검정고시
2교시
수학 정답표
문항번호 정 답 문항번호 정 답
1 ③ 11 ①
2 ① 12 ②
3 ④ 13 ④
4 ③ 14 ①
5 ③ 15 ④
6 ④ 16 ④
7 ② 17 ①
8 ③ 18 ③
9 ② 19 ①
10 ② 20 ②
2018년도 제1회 고등학교 졸업학력 검정고시
3교시
영어 정답표
문항번호 정 답 문항번호 정 답 문항번호 정 답
1 ④ 11 ① 21 ③
2 ④ 12 ② 22 ③
3 ① 13 ② 23 ③
4 ④ 14 ③ 24 ②
5 ② 15 ② 25 ①
6 ③ 16 ④
7 ① 17 ①
8 ④ 18 ④
9 ① 19 ③
10 ③ 20 ②
2018년도 제1회 고등학교 졸업학력 검정고시
4교시
사회 정답표
문항번호 정 답 문항번호 정 답 문항번호 정 답
1 ③ 11 ② 21 ①
2 ④ 12 ④ 22 ②
3 ② 13 ② 23 ④
4 ① 14 ③ 24 ④
5 ④ 15 ④ 25 ②
6 ③ 16 ③
7 ② 17 ①
8 ① 18 ③
9 ① 19 ①
10 ③ 20 ④
2018년도 제1회 고등학교 졸업학력 검정고시
5교시
과학 정답표
문항번호 정 답 문항번호 정 답 문항번호 정 답
1 ③ 11 ② 21 ②
2 ② 12 ④ 22 ①
3 ④ 13 ③ 23 ③
4 ① 14 ② 24 ③
5 ④ 15 ① 25 ②
6 ② 16 ②
7 ① 17 ①
8 ③ 18 ④
9 ④ 19 ④


# 문제json파일과 답지json파일 합치기

In [44]:
import json
import os

In [45]:
# 📌 파일 로드
questions_json = "./data/society_questions.json"
answers_json = "./data/society_answers.json"
output_json = "./data/society_dataset.json"

In [46]:
# ✅ 문제 파일 로드
with open(questions_json, "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# ✅ 정답 파일 로드
with open(answers_json, "r", encoding="utf-8") as f:
    answers_data = json.load(f)

In [169]:
# 🔹 파일명 정리 함수 (정답 파일명과 문제 파일명 일치하도록 변환)
def clean_filename(filename):
    return filename.replace('_정답.pdf', '_사회.pdf')  # 정답 파일명을 문제 파일명과 맞춤

In [170]:
clean_filename('2018_고졸학력_1차시험_정답.pdf')

'2018_고졸학력_1차시험_정답.pdf'

In [137]:
# 🔍 변환된 정답 데이터 키 값 수정
answers_data_fixed = {clean_filename(k)): v for k, v in answers_data.items()}

In [139]:
answers_data_fixed.keys()

dict_keys(['2018_고졸학력_1차시험_사회.pdf', '2018_고졸학력_2차시험_사회.pdf', '2019_고졸학력_1차시험_사회.pdf', '2019_고졸학력_2차시험_사회.pdf', '2020_고졸학력_1차시험_사회.pdf', '2020_고졸학력_2차시험_정답(확정안).pdf', '2021_고졸학력_1차시험_정답(확정안).pdf', '2021_고졸학력_2차시험_정답(확정안).pdf', '2022_고졸학력_1차시험_정답(확정안).pdf', '2022_고졸학력_2차시험_정답(확정안).pdf', '2023_고졸학력_1차시험_정답(확정안).pdf', '2023_고졸학력_2차시험_정답(확정안).pdf', '2024_고졸학력_1차시험_정답(확정안).pdf', '2024_고졸학력_2차시험_정답(확정안).pdf'])

In [143]:
questions_data.keys()

dict_keys(['2018_고졸학력_1차시험_사회.pdf', '2018_고졸학력_2차시험_사회.pdf', '2019_고졸학력_1차시험_사회.pdf', '2019_고졸학력_2차시험_사회.pdf', '2020_고졸학력_1차시험_사회.pdf', '2020_고졸학력_2차시험_사회.pdf', '2021_고졸학력_1차시험_사회.pdf', '2021_고졸학력_2차시험_사회.pdf', '2022_고졸학력_1차시험_사회.pdf', '2022_고졸학력_2차시험_사회.pdf', '2023_고졸학력_1차시험_사회.pdf', '2023_고졸학력_2차시험_사회.pdf', '2024_고졸학력_1차시험_사회.pdf', '2024_고졸학력_2차시험_사회.pdf'])

In [168]:
# ✅ 문제와 정답을 합쳐서 저장할 딕셔너리
merged_data = {}

# 🔥 문제와 정답 매칭
for file_name, question_content in questions_data.items():
    matched_file = clean_filename(normalize_filename(file_name))
    if matched_file in answers_data_fixed:  # 정답이 있는 경우만 추가
        merged_data[file_name] = {
            "questions": question_content,
            "answers": answers_data_fixed[matched_file]
        }
    else:
        print(f"⚠ 정답이 없는 문제 파일: {file_name}")

⚠ 정답이 없는 문제 파일: 2020_고졸학력_2차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2021_고졸학력_1차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2021_고졸학력_2차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2022_고졸학력_1차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2022_고졸학력_2차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2023_고졸학력_1차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2023_고졸학력_2차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2024_고졸학력_1차시험_사회.pdf
⚠ 정답이 없는 문제 파일: 2024_고졸학력_2차시험_사회.pdf


In [19]:
# ✅ JSON 파일 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=4)

print(f"✅ 문제 + 정답 JSON 저장 완료! ({output_json})")

✅ 문제 + 정답 JSON 저장 완료! (highschool_english_dataset.json)
